In [210]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [211]:
milk_df=pd.read_csv(r'..\Datasets\milk.csv',index_col=0)
print(milk_df.shape)
milk_df.head()

(25, 5)


,water,protein,fat,lactose,ash
Animal,,,,,
HORSE,90.1,2.6,1.0,6.9,0.35
ORANGUTAN,88.5,1.4,3.5,6.0,0.24
MONKEY,88.4,2.2,2.7,6.4,0.18
DONKEY,90.3,1.7,1.4,6.2,0.40
HIPPO,90.4,0.6,4.5,4.4,0.10


In [212]:
clf=IsolationForest(contamination=0.05,random_state=25)
clf.fit(milk_df)
pred=clf.predict(milk_df)
(pred)

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1, -1,  1,  1,  1,  1, -1,  1])

Amoung the above labels `-1` indicates suspected outliers

### Nutrients

In [213]:
nutrients=pd.read_csv(r'..\Datasets\nutrient.csv',index_col=0)
nutrients.head()

,energy,protein,fat,calcium,iron
Food_Item,,,,,
BEEF BRAISED,340,20,28,9,2.6
HAMBURGER,245,21,17,9,2.7
BEEF ROAST,420,15,39,7,2.0
BEEF STEAK,375,19,32,9,2.6
BEEF CANNED,180,22,10,17,3.7


In [214]:
clf=IsolationForest(contamination=0.05,random_state=25)
clf.fit(nutrients)
pred=clf.predict(nutrients)
nutrients['label']=pred
nutrients[nutrients['label']==-1]

,energy,protein,fat,calcium,iron,label
Food_Item,,,,,,
BEEF ROAST,420,15,39,7,2.0,-1
CLAMS CANNED,45,7,1,74,5.4,-1


### for glass dataset

In [215]:
glass_df=pd.read_csv(r'..\Cases\Glass Identification\Glass.csv')
glass_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [216]:
glass_tst_df=pd.read_csv(r'..\Cases\Glass Identification\tst_Glass.csv')
glass_tst_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.5321,14.0,0.00,0.34,70.23,0.001,6.7,1.23,0.000
1,1.5212,15.0,3.00,1.23,75.90,0.100,7.0,0.00,0.440
2,1.5112,13.0,3.50,2.30,73.00,3.400,14.0,2.30,0.220
3,1.5000,12.4,1.23,3.22,74.22,4.500,10.0,3.10,0.100
4,1.5200,13.0,2.40,0.34,71.22,3.200,9.0,1.44,0.001


In [217]:
X,y=glass_df.drop('Type',axis=1),glass_df['Type']

In [218]:
clf=IsolationForest(contamination='auto',random_state=25)
clf.fit(X)
pred=clf.predict(X)

In [219]:
df_out=pd.DataFrame({'Type':list(y.values),'label':list(pred)})
pd.crosstab(df_out['Type'],df_out['label'],margins=True)

label,-1,1,All
Type,,,
building_windows_float_processed,0,70,70
building_windows_non_float_processed,8,68,76
containers,5,8,13
headlamps,6,23,29
tableware,1,8,9
vehicle_windows_float_processed,0,17,17
All,20,194,214


In [220]:
glass_in=glass_df[df_out['label']!=-1]
glass_in.shape

(194, 10)

In [221]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
glass_in['Type']=le.fit_transform(glass_in['Type'])
X_d,y_d=glass_in.drop('Type',axis=1),glass_in['Type']

C:\Users\dai.STUDENTSDC\AppData\Local\Temp\ipykernel_9980\3820936452.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glass_in['Type']=le.fit_transform(glass_in['Type'])


## Train model with whole data

In [222]:
X.shape,y.shape

((214, 9), (214,))

In [223]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [224]:
from sklearn.metrics import accuracy_score,log_loss
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(random_state=25)
rf.fit(X_train,y_train)
y_pred1=rf.predict(X_test)
y_pred_proba1=rf.predict_proba(X_test)
print("Accuracy: ",accuracy_score(y_test,y_pred1))
print("Log Loss: ",log_loss(y_test,y_pred_proba1))

Accuracy:  0.7538461538461538
Log Loss:  0.5716075632327596


## Building model without outliers

In [225]:
X_d.shape,y_d.shape

((194, 9), (194,))

In [226]:
X_train_d,X_test_d,y_train_d,y_test_d=train_test_split(X_d,y_d,test_size=0.3,random_state=25,stratify=y_d)

In [227]:
rf1=RandomForestClassifier(random_state=25)
rf1.fit(X_train_d,y_train_d)
y_pred_d1=rf1.predict(X_test_d)
y_pred_proba_d1=rf.predict_proba(X_test_d)
print("Accuracy: ",accuracy_score(y_test_d,y_pred_d1))
print("Log Loss: ",log_loss(y_test_d,y_pred_proba_d1))

Accuracy:  0.7627118644067796
Log Loss:  0.3538339031201463


In [229]:
y_pred_d1=rf1.predict(glass_tst_df)
y_pred_proba_d1=rf.predict_proba(glass_tst_df)
y_pred_d1

array([3, 1, 3, 3, 3, 3])

## For Concrete Strength (Regression)

In [230]:
Concrete_Data_df=pd.read_csv(r'..\Cases\Concrete Strength\Concrete_Data.csv')
Concrete_Data_df.head()

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [235]:
x,y=Concrete_Data_df.drop('Strength',axis=1),Concrete_Data_df[['Strength']]
y

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30
...,...
1025,44.28
1026,31.18
1027,23.70
1028,32.77


In [236]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=25)

In [237]:
from sklearn.metrics import accuracy_score,log_loss,r2_score
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor(random_state=25)
rf.fit(X_train,y_train)
y_pred1=rf.predict(X_test)
# y_pred_proba1=rf.predict_proba(X_test)
print("r2_score: ",r2_score(y_test,y_pred1))

c:\Users\dai.STUDENTSDC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


r2_score:  0.8860376553315042
